In [33]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup


def remove_td(column):
    remove_one = column.split('<')
    remove_two = remove_one[0].split('>')
    return remove_two[1].replace(",", "")

def is_float(input):
    try:
        num = float(input)
    except ValueError:
        return False
    return True

def financial_statement_debug(year, season, type='綜合損益彙總表'):
    if year >= 1000:
        year -= 1911

    if type == '綜合損益彙總表':
        url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb04'
    elif type == '資產負債彙總表':
        url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb05'
    elif type == '營益分析彙總表':
        url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb06'
    else:
        print('type does not match')
        
     # 偽瀏覽器
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

    r = requests.post(url, {
        'encodeURIComponent':1,
        'step':1,
        'firstin':1,
        'off':1,
        'TYPEK':'sii',
        'year':str(year),
        'season':str(season),
    }, headers=headers)

    r.encoding = 'utf8'
    soup = BeautifulSoup(r.text, "lxml")
    tables = soup.find_all("table", class_ = "hasBorder")
    return tables

def financial_statement(year, season, type='綜合損益彙總表'):

    if year >= 1000:
        year -= 1911

    if type == '綜合損益彙總表':
        url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb04'
    elif type == '資產負債彙總表':
        url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb05'
    elif type == '營益分析彙總表':
        url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb06'
    else:
        print('type does not match')

    # 偽瀏覽器
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    
    r = requests.post(url, {
        'encodeURIComponent':1,
        'step':1,
        'firstin':1,
        'off':1,
        'TYPEK':'sii',
        'year':str(year),
        'season':str(season),
    }, headers=headers)

    r.encoding = 'utf8'
    
    soup = BeautifulSoup(r.text, "lxml")
    tables = soup.find_all("table", class_ = "hasBorder")
    cols = str(tables[2]).split('<tr')
    
    result_table = []
    header = []
    data_array = []
    
    tds = cols[1].split('<th')
    for j in range(1, len(tds)):
        #print(" ", remove_td(tds[j]))
        header.append(str(remove_td(tds[j])))
    
    for i in range(0, len(cols)):
        tds = cols[i].split('<td')
        data_array = []
        for j in range(1, len(tds)):
            #print(" ", remove_td(tds[j]))
            data_array.append(remove_td(tds[j]))
        result_table.append(data_array)
    
    df = pd.DataFrame.from_records(result_table)
    df.columns = header
    #df = df.set_index(['公司'])
    
    #用 .drop 方法刪除 Pandas Dataframe 中列值的行
    indexNames = df[ df['營業收入'] == '--' ].index
    df.drop(indexNames, inplace=True)
    
    try:
        # convert object type to int type
        df['營業收入'] = df['營業收入'].astype(float)
        df['營業毛利（毛損）淨額'] = df['營業毛利（毛損）淨額'].astype(float)
        df['營業利益（損失）'] = df['營業利益（損失）'].astype(float)
        df['稅前淨利（淨損）'] = df['稅前淨利（淨損）'].astype(float)
        df['本期淨利（淨損）'] = df['本期淨利（淨損）'].astype(float)
        df['基本每股盈餘（元）'] = df['基本每股盈餘（元）'].astype(float)

        # setup display float format
        pd.options.display.float_format = '${:,.2f}'.format
        # prepare rate info
        df2 = pd.DataFrame()
        df2['公司'] = df['公司']
        df2['公司名稱'] = df['公司名稱']
        df2['營業收入(百萬元)'] = df['營業收入'] / 1000
        df2['毛利率(%)'] = 100.0 * df['營業毛利（毛損）淨額'] / df['營業收入']
        df2['營業利益率(%)'] = 100.0 * df['營業利益（損失）'] / df['營業收入']
        df2['稅前純益率(%)'] = 100.0 * df['稅前淨利（淨損）'] / df['營業收入']
        df2['稅後純益率(%)'] = 100.0 * df['本期淨利（淨損）'] / df['營業收入']
        df2['基本每股盈餘（元）'] =  df['基本每股盈餘（元）']
        #df2 = df2.set_index(['公司名稱'])
        df2 = df2.set_index(['公司'])
    except Exception as e:
        print(e)
    return df2

def simple_financial_statement(year, season, type='綜合損益彙總表'):

    if year >= 1000:
        year -= 1911

    if type == '綜合損益彙總表':
        url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb04'
    elif type == '資產負債彙總表':
        url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb05'
    elif type == '營益分析彙總表':
        url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb06'
    else:
        print('type does not match')
        
    # 偽瀏覽器
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

    r = requests.post(url, {
        'encodeURIComponent':1,
        'step':1,
        'firstin':1,
        'off':1,
        'TYPEK':'sii',
        'year':str(year),
        'season':str(season),
    }, headers=headers)

    r.encoding = 'utf8'
    
    soup = BeautifulSoup(r.text, "lxml")
    tables = soup.find_all("table", class_ = "hasBorder")
    cols = str(tables[2]).split('<tr')
    
    result_table = []
    header = []
    data_array = []
    
    tds = cols[1].split('<th')
    for j in range(1, len(tds)):
        #print(" ", remove_td(tds[j]))
        header.append(str(remove_td(tds[j])))
    
    for i in range(0, len(cols)):
        tds = cols[i].split('<td')
        data_array = []
        for j in range(1, len(tds)):
            #print(" ", remove_td(tds[j]))
            data_array.append(remove_td(tds[j]))
        result_table.append(data_array)
    
    df = pd.DataFrame.from_records(result_table)
    df.columns = header
    #df = df.set_index(['公司'])
    
    #用 .drop 方法刪除 Pandas Dataframe 中列值的行
    indexNames = df[ df['營業收入'] == '--' ].index
    df.drop(indexNames, inplace=True)
    
    # convert object type to int type
    df['營業收入'] = df['營業收入'].astype(float)
    df['基本每股盈餘（元）'] = df['基本每股盈餘（元）'].astype(float)

    # setup display float format
    pd.options.display.float_format = '${:,.2f}'.format
    # prepare rate info
    df2 = pd.DataFrame()
    df2['公司'] = df['公司']
    df2['公司名稱'] = df['公司名稱']
    df2['營業收入(百萬元)'] = df['營業收入'] / 1000
    df2['基本每股盈餘（元）'] =  df['基本每股盈餘（元）']
    #df2 = df2.set_index(['公司名稱'])
    df2 = df2.set_index(['公司'])
    return df2

In [36]:
stock_debug = simple_financial_statement(110, 2)
print(stock_debug)

      公司名稱  營業收入(百萬元)  基本每股盈餘（元）
公司                              
NaN   None       $nan       $nan
NaN   None       $nan       $nan
1101    台泥 $50,023.36      $1.82
1102    亞泥 $39,961.67      $2.46
1103    嘉泥  $1,094.91     $-0.45
...    ...        ...        ...
9944    新麗  $1,475.78      $0.62
9945   潤泰新 $11,592.14      $7.16
9946  三發地產    $450.74      $0.16
9955    佳龍    $795.70     $-0.26
9958   世紀鋼  $4,099.35      $2.89

[915 rows x 3 columns]


In [43]:
stock = financial_statement(110,2)
print(stock)

      公司名稱  營業收入(百萬元)  毛利率(%)  營業利益率(%)  稅前純益率(%)  稅後純益率(%)  基本每股盈餘（元）
公司                                                                    
NaN   None       $nan    $nan      $nan      $nan      $nan       $nan
NaN   None       $nan    $nan      $nan      $nan      $nan       $nan
1101    台泥 $50,023.36  $28.25    $22.49    $29.29    $24.00      $1.82
1102    亞泥 $39,961.67  $26.45    $22.60    $28.90    $23.06      $2.46
1103    嘉泥  $1,094.91  $-0.92   $-27.07   $-25.75   $-25.08     $-0.45
...    ...        ...     ...       ...       ...       ...        ...
9944    新麗  $1,475.78  $28.88     $9.46     $9.01     $5.30      $0.62
9945   潤泰新 $11,592.14  $20.92    $13.00    $96.50    $94.09      $7.16
9946  三發地產    $450.74  $25.87    $11.89    $12.65    $11.08      $0.16
9955    佳龍    $795.70   $4.25    $-1.94    $-3.38    $-3.38     $-0.26
9958   世紀鋼  $4,099.35  $26.30    $21.14    $25.08    $20.34      $2.89

[915 rows x 7 columns]


In [38]:
cond1 = stock['稅後純益率(%)'] > 4.0
cond2 = stock['基本每股盈餘（元）'] > 2.0

#cond1 = stock['稅後純益率(%)'] > 5.0
#cond2 = stock['基本每股盈餘（元）'] > 3.0
ret_stock = stock[cond1 & cond2]

#cond1 = stock['毛利率(%)'] <= 20
#cond2 = stock['毛利率(%)'] > 10
#cond3 = stock['營業利益率(%)'] <= 20
#cond4 = stock['營業利益率(%)'] > 10
#stoc#k[cond1 & cond2 & cond3 & cond4]
#
#cond1 = stock['基本每股盈餘（元）'] > 3
#stock[cond1]
print(ret_stock.loc['1232'])
print(ret_stock.loc['1215'])

公司名稱              大統益
營業收入(百萬元)   $9,760.47
毛利率(%)         $13.85
營業利益率(%)        $9.51
稅前純益率(%)       $10.00
稅後純益率(%)        $7.92
基本每股盈餘（元）       $4.73
Name: 1232, dtype: object
公司名稱                卜蜂
營業收入(百萬元)   $11,828.22
毛利率(%)          $15.14
營業利益率(%)         $7.81
稅前純益率(%)         $8.13
稅後純益率(%)         $6.46
基本每股盈餘（元）        $2.91
Name: 1215, dtype: object


In [27]:
import twstock
twstock.codes['2379']

# 這是抓取歷史資料
stock_2330 = twstock.Stock('2330')

price_2330 = stock_2330.price[-5:]       # 近五日之收盤價
high_2330 = stock_2330.high[-5:]         # 近五日之盤中高點
low_2330 = stock_2330.low[-5:]           # 近五日之盤中低點
date_2330 = stock_2330.date[-5:]         # 近五日的日期

print('price_2330--->', price_2330)
print('high_2330--->', high_2330)
print('low_2330--->', low_2330)
print('date_2330--->', date_2330)

price_2330---> [290.0, 291.5, 294.0, 297.5, 292.0]
high_2330---> [293.5, 294.0, 294.0, 298.0, 295.0]
low_2330---> [290.0, 291.5, 291.0, 295.0, 291.0]
date_2330---> [datetime.datetime(2020, 5, 18, 0, 0), datetime.datetime(2020, 5, 19, 0, 0), datetime.datetime(2020, 5, 20, 0, 0), datetime.datetime(2020, 5, 21, 0, 0), datetime.datetime(2020, 5, 22, 0, 0)]


In [6]:
stock2 = financial_statement(110,2)

In [12]:
index = '2379'
curr_price = twstock.Stock(index).price[-1]
print(curr_price)

[247.0]


In [50]:
### Old version of calculating EPS ###
import time
import twstock
stock = financial_statement(110,2)
cond1 = stock['稅後純益率(%)'] > 4.0
cond2 = stock['基本每股盈餘（元）'] > 2.5
stock2 = stock[cond1 & cond2]

#cond1 = stock['毛利率(%)'] <= 20
#cond2 = stock['毛利率(%)'] > 10
#cond3 = stock['營業利益率(%)'] <= 20
#cond4 = stock['營業利益率(%)'] > 10
#stoc#k[cond1 & cond2 & cond3 & cond4]
#
#cond1 = stock['基本每股盈餘（元）'] > 3
#stock[cond1]
for index, row in stock2.iterrows():
    #get the current stock's price
    if len(twstock.Stock(index).price) > 1:
        curr_price = twstock.Stock(index).price[-1]
        eps = curr_price / (row['基本每股盈餘（元）'] * 2.0)
        print(row["公司名稱"], "每季基本每股盈餘（元）:", row['基本每股盈餘（元）'], "目前股價:", curr_price, "推估EPS:" + str(eps))
        time.sleep(10)

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [48]:
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as pdr
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False

df_2330 = pdr.DataReader('2330.TW', 'yahoo')

startTime = '2018-10-01'
endTime = '2020-5-14'
df_2330 = pdr.DataReader('2330.TW', 'yahoo', startTime, endTime)


fig = plt.figure(figsize=(10, 6))
plt.plot(df_2330.Close, '-' , label="收盤價")
plt.plot(df_2330.Open, '-' , label="開盤價")
plt.title('台積電2018 開盤/收盤價曲線',loc='right')
# loc->title的位置
plt.xlabel('日期')
plt.ylabel('收盤價')
plt.grid(True, axis='y')
plt.legend()

RemoteDataError: Unable to read URL: https://finance.yahoo.com/quote/2330.TW/history?period1=1476676800&period2=1634443199&interval=1d&frequency=1d&filter=history
Response Text:
b'<!DOCTYPE html>\n  <html lang="en-us"><head>\n  <meta http-equiv="content-type" content="text/html; charset=UTF-8">\n      <meta charset="utf-8">\n      <title>Yahoo</title>\n      <meta name="viewport" content="width=device-width,initial-scale=1,minimal-ui">\n      <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n      <style>\n  html {\n      height: 100%;\n  }\n  body {\n      background: #fafafc url(https://s.yimg.com/nn/img/sad-panda-201402200631.png) 50% 50%;\n      background-size: cover;\n      height: 100%;\n      text-align: center;\n      font: 300 18px "helvetica neue", helvetica, verdana, tahoma, arial, sans-serif;\n  }\n  table {\n      height: 100%;\n      width: 100%;\n      table-layout: fixed;\n      border-collapse: collapse;\n      border-spacing: 0;\n      border: none;\n  

In [71]:
import time
import twstock

stock = financial_statement(110, 2)
cond1 = stock['稅後純益率(%)'] > 5.0
cond2 = stock['基本每股盈餘（元）'] > 2.5
stock2 = stock[cond1 & cond2]

#cond1 = stock['毛利率(%)'] <= 20
#cond2 = stock['毛利率(%)'] > 10
#cond3 = stock['營業利益率(%)'] <= 20
#cond4 = stock['營業利益率(%)'] > 10
#stoc#k[cond1 & cond2 & cond3 & cond4]
#
#cond1 = stock['基本每股盈餘（元）'] > 3
#stock[cond1]

stock_list = []
for index, row in stock2.iterrows():
    stock_list.append(index)

twstock.realtime.mock = False  # 使用正常資料
realtime_stocks = twstock.realtime.get(stock_list)
print(realtime_stocks)
rtStcks = dict.copy(realtime_stocks)

def is_number(s, index):
    try:
        float(s.get(index).get('realtime').get('latest_trade_price'))
        return True
    except:
        return False

print("%-10s %-16s %-8s %-8s %-12s" % (
    "公司      ", "累積每股盈餘(元)", "目前股價", "預計整年盈餘(元)", "推估年本益比"))
for index, row in stock2.iterrows():
    #get the current stock's price
    if is_number(rtStcks, index):
        curr_price = float(rtStcks.get(index).get('realtime').get('latest_trade_price'))
    else:
        curr_price = 0
    
    if curr_price > 0:
        year_earning = row['基本每股盈餘（元）'] * 4.0
        eps = round(curr_price / year_earning, 2)
        print("%-10s %-16s %-8s %-8s %-12s" % (
            row["公司名稱"], str(row['基本每股盈餘（元）']), str(curr_price), str(year_earning), str(eps)))

{'rtcode': '9999', 'rtmessage': '參數不足', 'success': False}
公司         累積每股盈餘(元)        目前股價     預計整年盈餘(元) 推估年本益比      


In [72]:
twstock.realtime.get(stock_list)

{'rtcode': '9999', 'rtmessage': '參數不足', 'success': False}

In [64]:
twstock.realtime.mock = False  # 使用正常資料
ret = twstock.realtime.get(['2330'])
print(ret)
#data = ret.get('2330').get('realtime')
#print(data)

{'2330': {'timestamp': 1596004200.0, 'info': {'code': '2330', 'channel': '2330.tw', 'name': '台積電', 'fullname': '台灣積體電路製造股份有限公司', 'time': '2020-07-29 06:30:00'}, 'realtime': {'latest_trade_price': '422.0000', 'trade_volume': '4993', 'accumulate_trade_volume': '85236', 'best_bid_price': ['422.0000', '421.5000', '421.0000', '420.5000', '420.0000'], 'best_bid_volume': ['590', '867', '827', '765', '2472'], 'best_ask_price': ['422.5000', '423.0000', '423.5000', '424.0000', '424.5000'], 'best_ask_volume': ['39', '33', '33', '87', '139'], 'open': '428.0000', 'high': '431.5000', 'low': '419.5000'}, 'success': True}, 'success': True}


In [53]:
stock = twstock.Stock('1215')
bfp = twstock.BestFourPoint(stock)
ret = bfp.best_four_point_to_buy()   # 判斷是否為四大買點
print("To buy:", ret)
ret = bfp.best_four_point_to_sell()  # 判斷是否為四大賣點
print("To sell:", ret)
ret = bfp.best_four_point()          # 綜合判斷
print("Result:", ret)

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [59]:
stock = financial_statement(110,2)
cond1 = stock['稅後純益率(%)'] > 5.0
cond2 = stock['基本每股盈餘（元）'] > 2.5
stock2 = stock[cond1 & cond2]

stock_list = []
for index, row in stock2.iterrows():
    stock_list.append(index)
for index, row in stock2.iterrows():
    print(row["公司名稱"], str(row['基本每股盈餘（元）']))

大統益 4.73
鮮活果汁-KY 12.86
台塑 5.53
南亞 5.13
國喬 3.95
台化 4.32
富林-KY 3.91
華友聯 4.48
怡華 3.55
儒鴻 10.51
聚陽 5.17
廣隆 5.23
伸興 8.91
亞德客-KY 17.12
葡萄王 3.8
勝一 4.51
新光鋼 5.04
上銀 4.79
川湖 8.58
南帝 10.6
鑫永銓 16.34
中華 4.65
裕日車 5.01
宇隆 4.47
IKKA-KY 4.32
光寶科 3.57
台達電 5.46
國巨 22.95
台積電 10.57
智邦 3.74
海悅 7.6
華碩 28.49
致茂 6.5
凱美 4.45
技嘉 9.51
微星 10.14
瑞昱 14.41
台光電 6.65
群光 4.1
云辰 5.82
研華 4.79
漢唐 6.84
興勤 6.57
超豐 3.63
聯發科 33.65
義隆 8.3
麗臺 4.99
可成 4.19
美隆電 7.85
大毅 3.72
瑞軒 7.0
吉祥全 3.62
華新科 8.88
冠德 3.85
興富發 4.23
根基 3.77
潤弘 5.96
長榮 15.03
陽明 17.88
中航 5.2
萬海 15.19
台驊投控 16.62
美食-KY 3.6
八方雲集 4.71
潤泰全 19.16
神基 4.96
晶豪科 6.92
大立光 61.69
聯陽 5.21
奇鋐 3.78
信邦 5.25
禾伸堂 7.13
聯詠 25.74
晶技 5.01
健鼎 5.21
一零四 5.8
玉晶光 3.56
京鼎 8.03
創意 3.94
華擎 10.74
晶相光 4.76
嘉澤 14.91
敦泰 14.7
嘉威 5.25
牧德 9.03
健策 3.92
世芯-KY 11.22
貿聯-KY 6.48
圓展 4.3
新至陞 4.68
日月光投控 4.37
麗豐-KY 8.03
冠星-KY 6.48
拓凱 4.5
智伸科 5.52
鈞興-KY 4.34
台耀 16.7
南寶 5.12
聯德控股-KY 5.03
茂林-KY 4.55
嘉彰 3.56
天鈺 15.35
十銓 3.86
立積 8.04
科嘉-KY 5.65
祥碩 21.38
禾聯碩 5.95
豐祥-KY 5.65
崇越 5.88
松翰 3.97
中租-KY 7.11
瑞儀 5.82
巨路 3.96
盛

In [55]:
stock_list

['1215',
 '1232',
 '1256',
 '1301',
 '1303',
 '1308',
 '1309',
 '1312',
 '1325',
 '1326',
 '1341',
 '1436',
 '1456',
 '1476',
 '1477',
 '1515',
 '1537',
 '1558',
 '1583',
 '1590',
 '1707',
 '1726',
 '1762',
 '1773',
 '1795',
 '2006',
 '2014',
 '2015',
 '2031',
 '2049',
 '2059',
 '2108',
 '2114',
 '2204',
 '2227',
 '2233',
 '2250',
 '2301',
 '2308',
 '2327',
 '2330',
 '2345',
 '2348',
 '2352',
 '2357',
 '2360',
 '2373',
 '2375',
 '2376',
 '2377',
 '2379',
 '2383',
 '2385',
 '2390',
 '2395',
 '2404',
 '2408',
 '2409',
 '2428',
 '2441',
 '2451',
 '2454',
 '2456',
 '2458',
 '2465',
 '2472',
 '2474',
 '2477',
 '2478',
 '2480',
 '2481',
 '2489',
 '2491',
 '2492',
 '2514',
 '2520',
 '2542',
 '2545',
 '2546',
 '2597',
 '2603',
 '2609',
 '2612',
 '2615',
 '2636',
 '2637',
 '2707',
 '2723',
 '2753',
 '2915',
 '3005',
 '3006',
 '3008',
 '3014',
 '3017',
 '3022',
 '3023',
 '3026',
 '3030',
 '3034',
 '3037',
 '3042',
 '3044',
 '3056',
 '3090',
 '3130',
 '3167',
 '3257',
 '3376',
 '3406',
 '3413',
 

In [56]:
realtime_stocks = twstock.realtime.get(stock_list)

In [57]:
realtime_stocks

{'rtcode': '9999', 'rtmessage': '參數不足', 'success': False}

In [58]:
twstock.realtime.get(['1101','1232'])

{'1101': {'timestamp': 1634279400.0,
  'info': {'code': '1101',
   'channel': '1101.tw',
   'name': '台泥',
   'fullname': '台灣水泥股份有限公司',
   'time': '2021-10-15 06:30:00'},
  'realtime': {'latest_trade_price': '50.1000',
   'trade_volume': '656',
   'accumulate_trade_volume': '9705',
   'best_bid_price': ['50.0000', '49.9500', '49.9000', '49.8500', '49.8000'],
   'best_bid_volume': ['299', '320', '483', '362', '1159'],
   'best_ask_price': ['50.1000', '50.2000', '50.3000', '50.4000', '50.5000'],
   'best_ask_volume': ['11', '202', '269', '253', '466'],
   'open': '50.0000',
   'high': '50.2000',
   'low': '49.8000'},
  'success': True},
 '1232': {'timestamp': 1634279400.0,
  'info': {'code': '1232',
   'channel': '1232.tw',
   'name': '大統益',
   'fullname': '大統益股份有限公司',
   'time': '2021-10-15 06:30:00'},
  'realtime': {'latest_trade_price': '158.5000',
   'trade_volume': '4',
   'accumulate_trade_volume': '77',
   'best_bid_price': ['158.0000',
    '157.5000',
    '157.0000',
    '156.5000